In [11]:
import pandas as pd
import numpy as np
import warnings
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
import seaborn as sns
import scipy.stats as stats
import matplotlib.pyplot as plt
from sklearn.metrics import RocCurveDisplay, auc, roc_curve
from sklearn.metrics import roc_auc_score, accuracy_score
import plotly.express as px
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.experimental import enable_iterative_imputer  # noqa
from sklearn.impute import IterativeImputer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, roc_curve
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm

from sklearn import set_config
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder

from sksurv.datasets import load_gbsg2
from sksurv.preprocessing import OneHotEncoder
from sksurv.ensemble import RandomSurvivalForest

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

warnings.filterwarnings('ignore')

In [12]:
# Data Import
df = pd.read_csv('Diabetes.csv')

In [13]:
# Defining Date Variables
lab_dates = ['sBP_Date', 'BMI_Date','LDL_Date','HDL_Date','A1c_Date','TG_Date','FBS_Date','Total_Cholesterol_Lab_Date']
comorbid_dates = ['Depression_OnsetDate', 'HTN_OnsetDate', 'OA_OnsetDate', 'COPD_Date', 'Hypertension_Medications_First_Instance','Corticosteroids_first_instance']
DM_dates = ['DM_OnsetDate','DM_Onset_Revised','DM_Onset_Revised_1YrPrior']

# Changing dates into proper types
df[lab_dates] = df[lab_dates].astype('datetime64[ns]')
df[comorbid_dates] = df[comorbid_dates].astype('datetime64[ns]')
df[DM_dates] = df[DM_dates].astype('datetime64[ns]')


# Setting the start date as the minimum date of the columns
NONDM_dates = lab_dates + comorbid_dates
df['start_date'] = df[NONDM_dates].min(axis = 1)

# If a comorbid date or DM date is NaT, then replace with max date of that column, which is assumed to be the last date of the study
for date in comorbid_dates+DM_dates:
    df[date] = df[date].replace(pd.NaT, pd.Timestamp(df[date].max()))
    surv_time_var_name = date.removesuffix('_OnsetDate').removesuffix('_First_Instance').removesuffix('_first_instance').removesuffix('_Date') + '_survival_time'
    df[surv_time_var_name] = df[date] - df['start_date']
    df[surv_time_var_name] = df[surv_time_var_name].dt.days


In [14]:
df.head()

,Age_at_Exam,sBP,sBP_Date,BMI,BMI_Date,LDL,LDL_Date,HDL,HDL_Date,A1c,...,start_date,Depression_survival_time,HTN_survival_time,OA_survival_time,COPD_survival_time,Hypertension_Medications_survival_time,Corticosteroids_survival_time,DM_survival_time,DM_Onset_Revised_survival_time,DM_Onset_Revised_1YrPrior_survival_time
0,65.0,126.0,2013-06-11,31.0,2013-06-11,1.7,2013-06-14,1.1,2013-06-14,5.4,...,2013-06-11,749,749,751,765,749,749,749,714,349
1,62.0,135.0,2014-06-19,25.8,2014-10-17,2.5,2014-05-28,1.4,2014-05-28,5.8,...,2014-01-27,519,0,521,535,143,519,519,484,119
2,63.0,133.0,2012-07-31,30.9,2011-12-01,1.7,2012-06-01,NaN,NaT,6.1,...,2005-01-11,3822,3417,3824,87,0,3822,3822,3787,3422
3,51.0,136.0,2014-01-06,56.7,2014-01-06,2.8,2014-01-14,1.9,2014-01-14,6.0,...,2014-01-06,540,540,542,556,540,540,540,505,140
4,40.0,123.0,2015-06-12,33.1,2015-06-12,2.5,2015-06-24,1.2,2015-06-24,5.8,...,2013-06-28,0,732,734,748,732,732,732,697,332


In [15]:
# Look at records with missing MCAR Data
null_data_sBP = df[df['sBP'].isnull()]
null_data_HDL = df[df['HDL'].isnull()]

# Drop records with missing sBP since MCAR
df = df[df['sBP'].notnull()]

# Drop records with ONLY HDL missing, since HDL is MCAR.
mask = df['HDL'].isnull() & df.drop('HDL', axis=1).notnull().all(axis=1)
df = df[~mask]

In [16]:
df_s = df[df['DM_survival_time'] >= 0]

KeyError: 'DM_surv_time'

In [ ]:
df_s['Sex'] = df_s['Sex'].replace({'Female' : 0, 'Male': 1})
df_s['Diabetes'] = df_s['Diabetes'].replace({0 : False, 1: True})
df_s['Hypertension_Medications'] = df['Hypertension_Medications'].isnull().astype(int)
df_s['Corticosteroids'] = df['Corticosteroids'].isnull().astype(int)

In [ ]:
# Drop columns: 'sBP_Date', 'BMI_Date' and 24 other columns
df_s = df_s.drop(columns=['sBP_Date', 'BMI_Date', 'LDL_Date', 'HDL_Date', 'A1c_Date', 'TG_Date', 'FBS_Date', 
                          'FBS>DM', 'Total_Cholesterol_Lab_Date', 'DM_OnsetDate', 'Depression_OnsetDate', 
                          'HTN_OnsetDate', 'OA_OnsetDate', 'COPD_Date', 'Hypertension_Medications_First_Instance', 
                          'Corticosteroids_first_instance', 'leastO(A1c_Date)', 'leastO(DM_OnsetDate)', 'leastO(FBS_Date)', 'start_date',
                          'LeastOfAll', 'A1C_BEF_DM', 'FBS_BEF_DM', 'Patient_ID', 'DM_Onset_Revised', 'DM_Onset_Revised_1YrPrior',
                          'DM_Onset_Revised_survival_time', 'DM_Onset_Revised_1YrPrior_survival_time', 'DIABETES'])

df_s.head()


,Age_at_Exam,sBP,BMI,LDL,HDL,A1c,TG,FBS,Total_Cholesterol,Diabetes,...,Hypertension_Medications,Corticosteroids,Sex,Depression_surv_time,HTN_surv_time,OA_surv_time,COPD_surv_time,Hypertension_Medications_surv_time,Corticosteroids_surv_time,DM_surv_time
0,65.0,126.0,31.0,1.7,1.1,5.4,2.3,5.8,3.8,False,...,1,1,0,749,749,751,765,749,749,749
1,62.0,135.0,25.8,2.5,1.4,5.8,1.4,5.4,4.5,False,...,0,1,0,519,0,521,535,143,519,519
2,63.0,133.0,30.9,1.7,NaN,6.1,0.8,5.6,NaN,False,...,0,1,1,3822,3417,3824,87,0,3822,3822
3,51.0,136.0,56.7,2.8,1.9,6.0,1.0,6.0,5.2,False,...,1,1,0,540,540,542,556,540,540,540
4,40.0,123.0,33.1,2.5,1.2,5.8,1.0,5.2,4.1,False,...,1,1,0,0,732,734,748,732,732,732


In [17]:
num_rows, num_cols = df_s.shape
y = list()
for i in range(num_rows):
  yi= (df_s['Diabetes'].iloc[i], df_s['DM_survival_time'].iloc[i])
  y.append(yi)

X = df_s.drop(['DM_survival_time', 'Diabetes'], inplace=False, axis=1)
dt =  np.dtype([('Diabetes', bool), ('DM_survival_time', int)])
y = np.array(y, dtype=dt)

KeyError: 'DM_survival_time'

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

imp = IterativeImputer()
X_train = imp.fit_transform(X_train)
X_test = imp.transform(X_test)

In [ ]:
rsf = RandomSurvivalForest(n_estimators=100, min_samples_split=150, min_samples_leaf=100, max_depth=15, verbose=2, n_jobs=-1, random_state=42)
rsf.fit(X_train, y_train)

In [ ]:
rsf.score(X_test, y_test)

In [ ]:
pred_test = pd.Series(rsf.predict(X_test))

In [ ]:
pd.DataFrame(X_test).head()


In [ ]:
X_test_sorted = pd.DataFrame(X_test, columns=X.columns).dropna().sort_values(by=["A1c", "FBS"])
X_test_sel = pd.concat((X_test_sorted.head(5),X_test_sorted.tail(5)))
#X_test_sel = X_test_sorted.iloc[range(0, len(X_test_sorted), 200)]

In [ ]:
surv = rsf.predict_survival_function(X_test_sel, return_array=True)

for i, s in enumerate(surv):
    plt.step(rsf.unique_times_, s, where="post")
plt.title("Survival Probability for top 5 and bottom 5 A1c values in test set") 
#plt.title("Survival Probability across range of A1c values in test set") 
plt.ylabel("Survival probability")
plt.xlabel("Time in days")
plt.grid(True)

In [ ]:
surv = rsf.predict_cumulative_hazard_function(X_test_sel, return_array=True)

for i, s in enumerate(surv):
    plt.step(rsf.unique_times_, s, where="post")
plt.ylabel("Cumulative hazard")
plt.xlabel("Time in days")
plt.grid(True)

In [ ]:
from sklearn.inspection import permutation_importance

result = permutation_importance(rsf, X_test, y_test, n_repeats=15, random_state=42)

In [ ]:
pd.DataFrame(
    {
        k: result[k]
        for k in (
            "importances_mean",
            "importances_std",
        )
    },
    index=X.columns,
).sort_values(by="importances_mean", ascending=False)

In [ ]:
sqdiff = (y_test['DM_survival_int']-pred_test)**2
sbd = list()
for i in range(len(sqdiff)):
    if y_test['Diabetes'][i] == True:
        sbd.append(sqdiff[i])

plt.plot(range(len(sbd)),sbd)
plt.ylabel("Squared Difference in Survival Time")
plt.xlabel("Patient Index")
plt.show()

In [ ]:
absdiff = y_test['DM_survival_int']-pred_test
abd = list()
for i in range(len(absdiff)):
    if y_test['Diabetes'][i] == True:
        abd.append(absdiff[i])

plt.plot(range(len(abd)),abd)
plt.ylabel("True Survival Time - Predicted Survival Time")
plt.xlabel("Patient Index")
plt.show()

In [ ]:
col = np.where(y_test['Diabetes']==0, 'b', np.where(y_test['DM_survival_int'] < pred_test,'r','g'))

plt.scatter(y_test['DM_survival_int'], pred_test, c=col)
plt.xlabel("True Survival Time")
plt.ylabel("Predicted Survival Time")
ax = plt.gca()
ax.set_xlim([0, 1750])
ax.set_ylim([0, 1750])
plt.show()

In [ ]:
import statistics
print(statistics.mean(absdiff))
print(statistics.median(absdiff))

print(statistics.mean(sqdiff))
print(statistics.median(sqdiff))